In [35]:
import os
import pandas as pd
import nltk
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN, SMOTE, RandomOverSampler
import sklearn.linear_model as model
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
import sklearn
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
import nltk
from nltk.probability import FreqDist, DictionaryProbDist, ELEProbDist, sum_logs
from nltk.classify.api import ClassifierI

### Basic Logistic Regression

Use Michelle's method for importing data since I can't do it correctly:

In [36]:
train_data = np.genfromtxt('Data/training_data.txt',dtype='str')
test_data = np.genfromtxt('Data/test_data.txt',dtype='str')
#train_data = np.loadtxt('training_data.txt',  delimiter=' ')
#test_data = np.loadtxt('test_data.txt')
train_labels = train_data[0, :]
train_stars = train_data[1:, 0]
train_reviews = train_data[1:, 1:]

test_labels = test_data[0, :]
#test_stars = train_data[1:, 0]
test_reviews = test_data[1:, 0:]

trainX = train_reviews
trainY = train_stars
testX = test_reviews
#testY = test_stars

print("trainX = ", trainX.shape)
print("trainY = ", trainY.shape)
print("testX = ", testX.shape)


for i in range(0, len(trainX)):
    for j in range(0, len(trainX[0])):
        trainX[i][j] = int(trainX[i][j])

for i in range(0, len(testX)):
    for j in range(0, len(testX[0])):
        testX[i][j] = int(testX[i][j])

for i in range(0, len(trainY)):
        trainY[i] = int(trainY[i])
#for i in range(0, len(trainY)):
        #testY[i] = int(testY[i])

trainX =  (20000, 1000)
trainY =  (20000,)
testX =  (10000, 1000)


Define a function for classification error:

In [37]:
def class_error(y, y_pred):
    misclassified = 0
    for i in range(len(y)):
        if y[i] != y_pred[i]:
            misclassified += 1
    return float(misclassified) / len(y)

Split data into 80% training set and 20% test set.

In [38]:
num_rows = []
for i in range(len(trainY)):
    num_rows.append(i)
    
np.random.shuffle(num_rows)
train_indices = num_rows[:16000]
valid_indices = num_rows[16000:]

train_X = []
train_y = []
valid_X = []
valid_y = []

for index in train_indices:
    train_X.append(trainX[index])
    train_y.append(trainY[index])
    
for index in valid_indices:
    valid_X.append(trainX[index])
    valid_y.append(trainY[index])

With Michelle's code, it doesn't actually convert the array contents into integers because she tried to do it in place. This code does it correctly out of place:

In [39]:
train_X2 = []
for row in train_X:
    temp = []
    for element in row:
        temp.append(int(element))
    train_X2.append(temp)

valid_X2 = []
for row in valid_X:
    temp = []
    for element in row:
        temp.append(int(element))
    valid_X2.append(temp)
    
valid_y2 = []
for row in valid_y:
    valid_y2.append(int(row))
    
train_y2 = []
for row in train_y:
    train_y2.append(int(row))
    
test_X2 = []
for row in testX:
    temp = []
    for element in row:
        temp.append(int(element))
    test_X2.append(temp)

Convert everything back to a numpy array:

In [40]:
train_X2 = np.array(train_X2)
valid_X2 = np.array(valid_X2)
train_y2 = np.array(train_y2)
valid_y2 = np.array(valid_y2)
test_X2 = np.array(test_X2)

Fit the model:

In [41]:
logistic = model.LogisticRegression()
logistic.fit(train_X2, train_y2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Print the classification error on the training set:

In [42]:
class_error(logistic.predict(train_X2), train_y2)

0.1228125

Print the classification error on the validation set:

In [43]:
class_error(logistic.predict(valid_X2), valid_y2)

0.14925

Logistic regression works better than a random forest. Now, we want to combine the training and validation sets again so that we can train our model on all the data to prevent overfitting. We will then use this to predict on the test set:

In [44]:
all_X = np.concatenate((train_X2, valid_X2))
all_y = np.concatenate((train_y2, valid_y2))

In [45]:
logistic = model.LogisticRegression()
logistic.fit(all_X, all_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

A function for writing to the text file for the submission:

In [87]:
def writeToText(predictions, filename):
    array = [["Id","Prediction"]]
    index = 1
    for i in range(len(predictions)):
        predict = predictions[index - 1]
        array.append([index, int(predictions[index - 1])])
        index += 1
    f = open(filename, 'w')
    writer = csv.writer(f, delimiter=',', quotechar='|')
    writer.writerows(array)

In [47]:
writeToText(logistic.predict(test_X2), 'logistic.txt')

### Logistic Regression with TF-IDF

A function to convert the set into a TF-IDF matrix:

In [48]:
def TFIDF(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element)):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    tokenize = lambda doc: doc.lower().split(" ")
    sklearn_tfidf = TfidfVectorizer(norm='l2',min_df=0, use_idf=True, smooth_idf=False, sublinear_tf=True, tokenizer=tokenize)
    return sklearn_tfidf.fit_transform(documents)

In [49]:
def extract_features(X, labels):
    documents = []

    for element in X:
        temp = ''
        for i in range(len(element)):
            word = labels[i + 1] + ' '
            new_word = word * element[i]
            temp += (new_word)
        documents.append(temp)
        
    document_words = set(document)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in document_words)
    return features


Fit the model using the training data:

In [50]:
logistic = model.LogisticRegression()
logistic.fit(TFIDF(train_X2, train_labels), train_y2)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [51]:
class_error(logistic.predict(TFIDF(valid_X2, train_labels)), valid_y2)

0.1425

This model has a lower classification error than the basic regression. We will now undergo the same procedure as in the last section to get the test predictions:

In [52]:
logistic = model.LogisticRegression()
logistic.fit(TFIDF(all_X, train_labels), all_y)
writeToText(logistic.predict(TFIDF(test_X2, train_labels)), 'logisticTFIDF.txt')

### GridSearch

In [53]:
clf = SGDClassifier(loss='hinge', penalty='l2', random_state=42).fit(train_X2, train_y2)
parameters = {'alpha': (1e-5, 1e-6)}
gs_clf = GridSearchCV(clf, parameters, n_jobs=-1)
gs_clf = gs_clf.fit(TFIDF(train_X2, train_labels), train_y2)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/Users/michellezhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/U

In [54]:
gs_clf.best_params_

{'alpha': 1e-05}

clf.get_params()

### Naive Bayes

In [55]:
nb = MultinomialNB(alpha=50).fit(TFIDF(train_X2, train_labels), train_y2)

In [56]:
training_set = nltk.classify.apply_features(train_X2, train_labels)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
classifier_nb = nltk.NaiveBayesClassifier.train(train_X2)


In [ ]:
nb.get_params()

In [ ]:
class_error(nb.predict(TFIDF(valid_X2, train_labels)), valid_y2)

### SVM

In [ ]:
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(TFIDF(train_X2, train_labels), train_y2)

In [ ]:
class_error(sgd.predict(valid_TFIDF), valid_y2)

In [ ]:
sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42)
sgd.fit(TFIDF(all_X, train_labels), all_y)
writeToText(sgd.predict(TFIDF(test_X2, train_labels)), 'SGD_TFIDF.txt')

### Blend the based on equal weights for each one:

In [57]:
valid_TFIDF = TFIDF(valid_X2, train_labels)
train_TFIDF = TFIDF(train_X2, train_labels)
all_TFIDF = TFIDF(all_X, train_labels)
test_TFIDF = TFIDF(test_X2, train_labels)

In [58]:
logistic = model.LogisticRegression()
logistic.fit(train_TFIDF, train_y2)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(train_TFIDF, train_y2)

nb = MultinomialNB(alpha=50).fit(train_TFIDF, train_y2)

pred1 = logistic.predict(valid_TFIDF)
pred2 = sgd.predict(valid_TFIDF)
pred3 = nb.predict(valid_TFIDF)

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [59]:
pred = []
for i in range(len(pred1)):
    
    if pred1[i] == pred2[i]:
        pred.append(pred1[i])
    elif pred1[i] == pred3[i]:
        pred.append(pred1[i])
    else:
        pred.append(pred2[i])

In [60]:
class_error(pred, valid_y2)

0.14575

In [61]:
logistic = model.LogisticRegression()
logistic.fit(all_TFIDF, all_y)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(all_TFIDF, all_y)

nb = MultinomialNB(alpha=50).fit(all_TFIDF, all_y)

pred1 = logistic.predict(test_TFIDF)
pred2 = sgd.predict(test_TFIDF)
pred3 = nb.predict(test_TFIDF)

pred = []
for i in range(len(pred1)):
    
    if pred1[i] == pred2[i]:
        pred.append(pred1[i])
    elif pred1[i] == pred3[i]:
        pred.append(pred1[i])
    else:
        pred.append(pred2[i])

writeToText(pred, 'Blend_TFIDF.txt')

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


### BernoulliNB

For this, we need binary features.

In [ ]:
train_X2[train_X2 > 1] = 1
valid_X2[valid_X2 > 1] = 1
all_X[all_X > 1] = 1

In [ ]:
bnb = BernoulliNB(alpha=1.0e-10)
bnb.fit(train_X2, train_y2)
class_error(bnb.predict(valid_X2), valid_y2)

# Multinomial NB

In [ ]:
mnb = MultinomialNB(alpha=50).fit(train_TFIDF, train_y2)
mnb.fit(train_X2, train_y2)
class_error(mnb.predict(valid_X2), valid_y2)

In [ ]:
mnb = MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False)
#gnb = gnb.fit(train_X2, trainY_1D)
print(len(all_trainX))
print(len(all_trainY))
mnb = mnb.fit(all_trainX, all_trainY)

### Probability Blend

Each model outputs can output a probability rather than a 0 or 1. We should use these probabilities when classifying the points.

In [ ]:
logistic = model.LogisticRegression()
logistic.fit(train_TFIDF, train_y2)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(train_TFIDF, train_y2)

nb = MultinomialNB(alpha=50).fit(train_TFIDF, train_y2)

In [ ]:
nb.predict_proba(valid_TFIDF)

In [ ]:
sgd.decision_function(valid_TFIDF)

In [ ]:
valid_y2

In [ ]:
logistic.decision_function(valid_TFIDF)

sgd and logistic have decision functions, whereas bayes has a probability. Since sgd and logistic are better models, we will look at when the magnitude of their confidence scores is greater than 1. If they both have magnitudes greater than 1 but opposite signs, or if the magnitude for both is less than 1, we get the majority vote from all 3 models.

In [102]:
def tieBreak(i, pred1, pred2, pred3):
    winner = 0
    if pred1[i] == pred2[i]:
        winner = pred1[i]
    elif pred1[i] == pred3[i]:
        winner = pred1[i]
    else:
        winner = pred2[i]
    return winner

In [ ]:
sgd_confidence = sgd.decision_function(valid_TFIDF)
log_confidence = logistic.decision_function(valid_TFIDF)
pred1 = sgd.predict(valid_TFIDF)
pred2 = logistic.predict(valid_TFIDF)
pred3 = nb.predict(valid_TFIDF)

pred_blend = []
for i in range(len(pred1)):
    if abs(sgd_confidence[i]) > 1 and abs(log_confidence[i]) < 1:
        pred_blend.append(pred1[i])
    elif abs(log_confidence[i]) > 1 and abs(sgd_confidence[i]) < 1:
        pred_blend.append(pred2[i])
    elif abs(log_confidence[i]) > 1 and abs(sgd_confidence[i]) > 1:
        if (log_confidence[i] > 0 and sgd_confidence[i] > 0) or (log_confidence[i] < 0 and sgd_confidence[i] < 0):
            pred_blend.append(pred1[i])
        else:
            pred_blend.append(tieBreak(i, pred1, pred2, pred3))
    else:
        pred_blend.append(tieBreak(i, pred1, pred2, pred3))


In [ ]:
class_error(pred_blend, valid_y2)

### Linear SVC

In [ ]:
lin = LinearSVC(random_state=0, loss='hinge', C=2)
lin.fit(train_TFIDF, train_y2)

In [ ]:
class_error(lin.predict(valid_TFIDF), valid_y2)

### Ridge

In [ ]:
ridge = RidgeClassifier(alpha=15, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
ridge.fit(train_TFIDF, train_y2)

In [ ]:
class_error(ridge.predict(valid_TFIDF), valid_y2)

### New Weighted Blends

In [ ]:
def tieBreak2(i, pred1, pred2, pred3, pred4, pred5):
    preds = [pred1, pred2, pred3, pred4, pred5]
    zeros = 0
    for element in preds:
        if element[i] == 0:
            zeros += 1
    if zeros > 2:
        return 0
    else:
        return 1

In [ ]:
logistic = model.LogisticRegression()
logistic.fit(train_TFIDF, train_y2)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(train_TFIDF, train_y2)

lin = LinearSVC(random_state=0, loss='hinge', C=2)
lin.fit(train_TFIDF, train_y2)

ridge = RidgeClassifier(alpha=15, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
ridge.fit(train_TFIDF, train_y2)

nb = MultinomialNB(alpha=50).fit(train_TFIDF, train_y2)

sgd_confidence = sgd.decision_function(valid_TFIDF)
log_confidence = logistic.decision_function(valid_TFIDF)
lin_confidence = lin.decision_function(valid_TFIDF)
ridge_confidence = ridge.decision_function(valid_TFIDF)

pred1 = sgd.predict(valid_TFIDF)
pred2 = logistic.predict(valid_TFIDF)
pred3 = nb.predict(valid_TFIDF)
pred4 = lin.predict(valid_TFIDF)
pred5 = ridge.predict(valid_TFIDF)

pred_blend = []
for i in range(len(pred1)):
    if abs(lin_confidence[i]) > 1 and abs(log_confidence[i]) < 1:
        pred_blend.append(pred1[i])
    elif abs(log_confidence[i]) > 1 and abs(lin_confidence[i]) < 1:
        pred_blend.append(pred2[i])
    elif abs(log_confidence[i]) > 1 and abs(lin_confidence[i]) > 1:
        if (log_confidence[i] > 0 and lin_confidence[i] > 0) or (log_confidence[i] < 0 and lin_confidence[i] < 0):
            pred_blend.append(pred1[i])
        else:
            pred_blend.append(tieBreak(i, pred4, pred2, pred3))
    else:
        pred_blend.append(tieBreak(i, pred4, pred2, pred3))

In [ ]:
class_error(pred_blend, valid_y2)

In [34]:
from tflearn.data_utils import to_categorical, pad_sequences
# from tflearn.datasets import imdb
import tensorflow as tf 
import keras
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import BatchNormalization

Using TensorFlow backend.


In [114]:
def neuralNetwork(train_X2, train_y2, testX):
    train_y = to_categorical(train_y2, 2)
    
    model = Sequential()

    keras.layers.BatchNormalization(axis=-1, 
    momentum=0.99, epsilon=0.001, center=True, 
    scale=True, beta_initializer='zeros', 
    gamma_initializer='ones', moving_mean_initializer='zeros', 
    moving_variance_initializer='ones', beta_regularizer=None, 
    gamma_regularizer=None, beta_constraint=None, gamma_constraint=None)

    # change input dim to 1000 for 
    model.add(Dense(200, input_dim=1000))
    # keras.layers.LeakyReLU(alpha=0.1)
    model.add(Activation('sigmoid'))
    model.add(Dropout(0.10))

    model.add(BatchNormalization())


    model.add(Dense(200)) 
    # model.add(BatchNormalization())
    model.add(Activation('sigmoid'))
    # keras.layers.LeakyReLU(alpha=0.1)
    model.add(Dropout(0.10))
    model.add(BatchNormalization())

    # model.add(Dense(333)) 
    # model.add(BatchNormalization())
    # model.add(Activation('sigmoid'))
    # # keras.layers.LeakyReLU(alpha=0.3)
    # model.add(Dropout(0.10))

    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.summary()

    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    # with tf idf

    #fit = model.fit(train_XTF[:, :1000], train_y2, batch_size=128, nb_epoch=10, validation_data=(valid_XTF, valid_y2),
        #verbose=1)


    # without tfidf
    fit = model.fit(train_X2[:, :1000], train_y, batch_size=150, nb_epoch=10, verbose=1)
    
    prediction = model.predict(testX[:, :1000])
    
    return prediction

It seems that a combination of linearSVC and LogisticRegression with Naive Bayes as the tie break works the best.

In [122]:
logistic = model.LogisticRegression()
logistic.fit(all_TFIDF, all_y)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(all_TFIDF, all_y)

lin = LinearSVC(random_state=0, loss='hinge', C=2)
lin.fit(all_TFIDF, all_y)

ridge = RidgeClassifier(alpha=15, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
ridge.fit(all_TFIDF, all_y)

nb = MultinomialNB(alpha=50).fit(all_TFIDF, all_y)

sgd_confidence = sgd.decision_function(test_TFIDF)
log_confidence = logistic.decision_function(test_TFIDF)
lin_confidence = lin.decision_function(test_TFIDF)
ridge_confidence = ridge.decision_function(test_TFIDF)

pred1 = sgd.predict(test_TFIDF)
pred2 = logistic.predict(test_TFIDF)
pred3 = nb.predict(test_TFIDF)
pred4 = lin.predict(test_TFIDF)
pred5 = ridge.predict(test_TFIDF)
pred_n = neuralNetwork(all_TFIDF, all_y, test_TFIDF)

pred_neural = []
for i in range(0, len(pred_n)):
    if abs(pred_n[i][0]-1) < abs(pred_n[i][1]-1):
        pred_neural.append(0)
    else:
        pred_neural.append(1)
        

pred_blend = []
pred_blend_uncertain = []
count = 0

for i in range(len(pred1)):
    if (log_confidence[i] > 1 and sgd_confidence[i] > 1) or (log_confidence[i] < -1 and sgd_confidence[i] < -1):
        pred_blend.append(pred1[i])
        count += 1
    else:
        pred_blend.append(pred_neural[i])
    

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 200)               200200    
_________________________________________________________________
activation_58 (Activation)   (None, 200)               0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 200)               0         
_________________________________________________________________
batch_normalization_59 (Batc (None, 200)               800       
_________________________________________________________________
dense_59 (Dense)             (None, 200)               40200     
_________________________________________________________________
activation_59 (Activation)   (None, 200)               0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 200)               0         
__________

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/keras/models.py:944: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/10
20000/20000 [==============================] - 3s 133us/step - loss: 0.7479 - acc: 0.5849
Epoch 2/10
20000/20000 [==============================] - 1s 49us/step - loss: 0.5068 - acc: 0.7527
Epoch 3/10
20000/20000 [==============================] - 1s 49us/step - loss: 0.4274 - acc: 0.8011
Epoch 4/10
20000/20000 [==============================] - 1s 49us/step - loss: 0.3919 - acc: 0.8243
Epoch 5/10
20000/20000 [==============================] - 1s 49us/step - loss: 0.3734 - acc: 0.8356
Epoch 6/10
20000/20000 [==============================] - 1s 50us/step - loss: 0.3559 - acc: 0.8425
Epoch 7/10
20000/20000 [==============================] - 1s 50us/step - loss: 0.3508 - acc: 0.8447
Epoch 8/10
20000/20000 [==============================] - 1s 49us/step - loss: 0.3458 - acc: 0.8470
Epoch 9/10
20000/20000 [==============================] - 1s 51us/step - loss: 0.3386 - acc: 0.8530
Epoch 10/10
20000/20000 [==============================] - 1s 50us/step - loss: 0.3329 - acc: 0.856

In [120]:
print((pred_neural))

[1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

In [125]:
print(pred_n)

[[ 0.03516929  0.96483076]
 [ 0.05394574  0.94605422]
 [ 0.67866564  0.32133433]
 ..., 
 [ 0.99435329  0.00564667]
 [ 0.01815656  0.98184341]
 [ 0.96672541  0.03327459]]


In [94]:
writeToText(pred_blend, 'Blend_TFIDF_Log_Neural.txt')

### K Neighbors Classifier

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=1000)
neigh.fit(train_TFIDF, train_y2) 

In [ ]:
class_error(neigh.predict(valid_TFIDF), valid_y2)

In [106]:
logistic = model.LogisticRegression()
logistic.fit(all_TFIDF, all_y)

sgd = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, random_state=42)
sgd.fit(all_TFIDF, all_y)

lin = LinearSVC(random_state=0, loss='hinge', C=2)
lin.fit(all_TFIDF, all_y)

ridge = RidgeClassifier(alpha=15, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)
ridge.fit(all_TFIDF, all_y)

nb = MultinomialNB(alpha=50).fit(all_TFIDF, all_y)

sgd_confidence = sgd.decision_function(test_TFIDF)
log_confidence = logistic.decision_function(test_TFIDF)
lin_confidence = lin.decision_function(test_TFIDF)
ridge_confidence = ridge.decision_function(test_TFIDF)

pred1 = sgd.predict(test_TFIDF)
pred2 = logistic.predict(test_TFIDF)
pred3 = nb.predict(test_TFIDF)
pred4 = lin.predict(test_TFIDF)
pred5 = ridge.predict(test_TFIDF)
#pred_n = neuralNetwork(all_TFIDF, all_y, test_TFIDF)
pred_blend2 = []

for i in range(len(pred1)):
    if abs(lin_confidence[i]) > 1 and abs(log_confidence[i]) < 1:
        pred_blend2.append(pred1[i])
    elif abs(log_confidence[i]) > 1 and abs(lin_confidence[i]) < 1:
        pred_blend2.append(pred2[i])
    elif abs(log_confidence[i]) > 1 and abs(lin_confidence[i]) > 1:
        if (log_confidence[i] > 0 and lin_confidence[i] > 0) or (log_confidence[i] < 0 and lin_confidence[i] < 0):
            pred_blend2.append(pred1[i])
        else:
            pred_blend2.append(tieBreak(i, pred4, pred2, pred3))
    else:
        pred_blend2.append(tieBreak(i, pred4, pred2, pred3))

/Users/michellezhao/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


In [107]:
print(pred_blend)


[1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

In [108]:
print(pred_blend2)

[1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 

In [118]:
wrong = 0
for i in range(0, len(pred_blend)):
    if pred_blend[i]!= pred_blend2[i]:
        wrong += 1
        
print(wrong)

285


In [121]:
wrong = 0
for i in range(0, len(pred_blend)):
    if pred_blend[i]!= pred_neural[i]:
        wrong += 1
        
print(wrong)

0


In [123]:
print(count)

6073


In [124]:
print(pred_n)

[[ 0.03516929  0.96483076]
 [ 0.05394574  0.94605422]
 [ 0.67866564  0.32133433]
 ..., 
 [ 0.99435329  0.00564667]
 [ 0.01815656  0.98184341]
 [ 0.96672541  0.03327459]]
